# 0.导入相关模块并定义全局变量

In [91]:
from os import listdir
from os.path import isfile,join
import re 
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_columns', 200)

path = r'/home/duhan/data/商品期货1分钟201509csv'
filenamePattern = r'(?P<code>\D+)(?P<num>\d+).csv' 
header=['date', 'time', 'open', 'high', 'low', 'close', 'lot', 'volume']

# 1.读取目录中的所有文件名列表

In [3]:
filenameList = [ filename for filename in listdir(path) if isfile(join(path,filename))  and re.match(filenamePattern, filename)]
filenameDataset = DataFrame(filenameList, columns=['name'])
filenameDataset['code'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('code'))
filenameDataset['num'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('num'))
filenameDataset.head()

,name,code,num
0,ZZOI13.csv,ZZOI,13
1,DLJM12.csv,DLJM,12
2,DLV13.csv,DLV,13
3,SQAG01.csv,SQAG,01
4,DLCS05.csv,DLCS,05


In [42]:
re.findall(r'\d+' , test) 

['13']

# 2. 获取所有品种列表

In [121]:
instrumentList  = list(filenameDataset.groupby('code').count().index)
print instrumentList
filenameDataset.groupby('code').count().head(10)

['DLAX', 'DLAY', 'DLB', 'DLBB', 'DLC', 'DLCS', 'DLFB', 'DLI', 'DLJ', 'DLJD', 'DLJM', 'DLL', 'DLM', 'DLP', 'DLPP', 'DLV', 'DLY', 'SQAG', 'SQAL', 'SQAU', 'SQBUX', 'SQBUY', 'SQCU', 'SQFU', 'SQHC', 'SQNI', 'SQPB', 'SQRB', 'SQRU', 'SQSN', 'SQWR', 'SQZN', 'ZZCF', 'ZZFG', 'ZZJR', 'ZZLR', 'ZZMA', 'ZZME', 'ZZOI', 'ZZPM', 'ZZRI', 'ZZRM', 'ZZRS', 'ZZSF', 'ZZSM', 'ZZSRX', 'ZZSRY', 'ZZTA', 'ZZTC', 'ZZWH']


,name,num
code,,
DLAX,7,7
DLAY,4,4
DLB,5,5
DLBB,5,5
DLC,7,7
DLCS,6,6
DLFB,7,7
DLI,13,13
DLJ,11,11


# 3. 定义通过文名件导入数据集的方法

In [ ]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [num]*8, df.columns ]
    return df


In [78]:
loadDataFile('DLAX05.csv').head()

05                                          
                        date   time  open  high   low close lot volume
2015-09-01 09:01  2015-09-01  09:01  4307  4307  4305  4307  24  33522
2015-09-01 09:02  2015-09-01  09:02  4304  4305  4303  4305  36  33512
2015-09-01 09:03  2015-09-01  09:03  4303  4303  4303  4303  16  33502
2015-09-01 09:04  2015-09-01  09:04  4302  4302  4299  4299  38  33498
2015-09-01 09:05  2015-09-01  09:05  4299  4299  4295  4296  56  33494

# 3.定义处理一种品种数据的函数

In [103]:
def loadInstrumentData(code):
    """
    给定顶一个品种代码，导入该品种的所有数据文件
    """
    c1 = filenameDataset['code'] == code
    df = pd.concat([ loadDataFile(filename) for filename in filenameDataset[ c1 ]['name'] ] , axis=1)
    df = df.sort_index(axis=1)
    return df

In [122]:
loadInstrumentData('DLI').head(10)

01                                                 \
                  close        date   high    lot    low   open   time   
2015-09-01 09:01  388.0  2015-09-01  389.0  25296  387.5  388.0  09:01   
2015-09-01 09:02  387.5  2015-09-01  388.0   3084  387.5  388.0  09:02   
2015-09-01 09:03  389.0  2015-09-01  389.0   7920  387.5  388.0  09:03   
2015-09-01 09:04  388.5  2015-09-01  389.0   1692  388.5  389.0  09:04   
2015-09-01 09:05  390.5  2015-09-01  390.5  29378  388.5  389.0  09:05   
2015-09-01 09:06  390.0  2015-09-01  390.5  17754  389.5  390.5  09:06   
2015-09-01 09:07  392.0  2015-09-01  392.5  48900  390.0  390.5  09:07   
2015-09-01 09:08  391.0  2015-09-01  392.0  32942  391.0  392.0  09:08   
2015-09-01 09:09  392.5  2015-09-01  392.5  21792  391.0  391.5  09:09   
2015-09-01 09:10  392.5  2015-09-01  393.0  35434  392.0  392.0  09:10   

                             02                                        03  \
                   volume close date high lot low open time volume  close   
2015-09-01 09:01  1601208   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:02  1601368   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:03  1600544   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:04  1600168   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:05  1604256   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:06  1604662   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN  371.0   
2015-09-01 09:07  1604756   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN  372.5   
2015-09-01 09:08  1613318   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:09  1619888   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   
2015-09-01 09:10  1622258   NaN  NaN  NaN NaN NaN  NaN  NaN    NaN    NaN   

                                                                       04  \
                        date   high lot    low   open   time volume close   
2015-09-01 09:01         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:02         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:03         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:04         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:05         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:06  2015-09-01  371.5   4  371.0  371.5  09:06    144   NaN   
2015-09-01 09:07  2015-09-01  372.5  10  371.5  371.5  09:07    154   NaN   
2015-09-01 09:08         NaN    NaN NaN    NaN    NaN    NaN    NaN   369   
2015-09-01 09:09         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   
2015-09-01 09:10         NaN    NaN NaN    NaN    NaN    NaN    NaN   NaN   

                                                                  05  \
                        date high lot  low open   time volume  close   
2015-09-01 09:01         NaN  NaN NaN  NaN  NaN    NaN    NaN  367.0   
2015-09-01 09:02         NaN  NaN NaN  NaN  NaN    NaN    NaN  367.0   
2015-09-01 09:03         NaN  NaN NaN  NaN  NaN    NaN    NaN  368.0   
2015-09-01 09:04         NaN  NaN NaN  NaN  NaN    NaN    NaN  368.5   
2015-09-01 09:05         NaN  NaN NaN  NaN  NaN    NaN    NaN  369.0   
2015-09-01 09:06         NaN  NaN NaN  NaN  NaN    NaN    NaN  369.5   
2015-09-01 09:07         NaN  NaN NaN  NaN  NaN    NaN    NaN  371.0   
2015-09-01 09:08  2015-09-01  369   2  369  369  09:08    242  370.5   
2015-09-01 09:09         NaN  NaN NaN  NaN  NaN    NaN    NaN  372.0   
2015-09-01 09:10         NaN  NaN NaN  NaN  NaN    NaN    NaN  371.5   

                                                                           06  \
                        date   high   lot    low   open   time  volume  close   
2015-09-01 09:01  2015-09-01  368.5  1662  367.0  367.5  09:01  244790  363.5   
2015-09-01 09:02  2015-09-01  367.5   146  367.0  367.0  09:02  244834    NaN   
2015-09-01 09:03  2015-09-01  368.0   994  367.0  367.0  09:03  245318    NaN   
2015-09-01 0

In [80]:
loadInstrumentData('ZZSM')

,name,code,num
26,ZZSM01.csv,ZZSM,01
384,ZZSM13.csv,ZZSM,13


In [76]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [num]*8, df.columns ]
    return df

07                                          
                        date   time  open  high   low close lot volume
2015-09-21 22:34  2015-09-21  22:34  4204  4204  4204  4204   8      6
2015-09-29 09:01  2015-09-29  09:01  4012  4191  4012  4191   4      8
2015-09-29 10:06  2015-09-29  10:06  4158  4158  4158  4158   2      6

In [71]:
    df =  pd.read_csv( join(path, 'DLAX07.csv'), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'],x['time']) , axis =1 )
    df.columns = [ ['01']*8, df.columns ]
    df 
    
    
    

01                                          
                        date   time  open  high   low close lot volume
2015-09-21 22:34  2015-09-21  22:34  4204  4204  4204  4204   8      6
2015-09-29 09:01  2015-09-29  09:01  4012  4191  4012  4191   4      8
2015-09-29 10:06  2015-09-29  10:06  4158  4158  4158  4158   2      6

In [50]:
help(df.apply)


Help on method apply in module pandas.core.frame:

apply(self, func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds) method of pandas.core.frame.DataFrame instance
    Applies function along input axis of DataFrame.
    
    Objects passed to functions are Series objects having index
    either the DataFrame's index (axis=0) or the columns (axis=1).
    Return type depends on whether passed function aggregates, or the
    reduce argument if the DataFrame is empty.
    
    Parameters
    ----------
    func : function
        Function to apply to each column/row
    axis : {0 or 'index', 1 or 'columns'}, default 0
        * 0 or 'index': apply function to each column
        * 1 or 'columns': apply function to each row
    broadcast : boolean, default False
        For aggregation functions, return object of same size with values
        propagated
    reduce : boolean or None, default None
        Try to apply reduction procedures. If the DataFrame is empty,
        